In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
import sys
sys.path.append("../model")
from Tip_adapter import TipAdapter
from dataset import FlickrAndPairs, TestMeta, TaskAWithLabel
import torch

In [86]:
data = FlickrAndPairs(path="/data4/saland/data/real_fake_pairs_1000_name.pt",load_from_disk=True)
# data = TestMeta("/data4/saland/data/test_meta.pt",load_from_disk=True)
tip = TipAdapter(100, data)

In [87]:
test_meta = TestMeta(path="/data4/saland/data/test_meta.pt",load_from_disk=True)
tip.get_accuracy(test_meta.features,test_meta.label,"cpu")

0.7576740980148315

In [88]:
taskA = TaskAWithLabel(path_to_csv="../../misc/scanFinal.csv",
                       path_to_taskA="/data4/saland/data/taskA.pt")

tip.get_accuracy(taskA.features,taskA.label,"cpu")

100%|██████████| 10080/10080 [00:05<00:00, 1976.29it/s]


0.6840277910232544

In [93]:
for k in (1,2,3,4,8,16,32,64,100,500,1000):
    tip = TipAdapter(k,data)
    print(f"accuracy on taskA    with cache of size {len(tip)}:",tip.get_accuracy(taskA.features,taskA.label,"cpu"))
    print(f"accuracy on testMeta with cache of size {len(tip)}:",tip.get_accuracy(test_meta.features,test_meta.label,"cpu"))